# Attention Architecture

Looks like transformer requires really large bptt, otherwise it seems to diverge

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.text import *

In [3]:
bs = 32
bptt = 128

In [4]:
# PATH=Path('data/wikitext-2-raw')
# data = TextLMDataBunch.load(PATH, cache_name='tmp_200', bs=bs, bptt=bptt)

In [5]:
PATH=Path('data/wikitext-2')
data = TextLMDataBunch.load(PATH, cache_name='tmp_tok_28', bs=bs, bptt=bptt)

In [6]:
data.show_batch()

idx,text
0,"xxbos xxmaj xxunk xxunk xxunk in xxunk , the xxmaj xxunk xxmaj xxunk xxunk xxunk to a xxunk xxunk of xxunk and xxunk xxunk in xxunk xxunk , xxunk xxunk xxunk xxunk xxunk xxunk , xxunk xxunk and xxunk . xxmaj xxunk xxunk xxunk , xxmaj xxunk xxmaj xxunk and the xxunk xxunk and xxunk xxunk a xxunk of xxunk xxunk xxunk the xxunk xxunk xxunk xxunkxx , xxunk xxunk"
1,"xxunk xxunk xxunk xxunkxx xxunk xxmaj xxunk xxmaj xxunk xxunk xxmaj xxunk , xxunk a xxunk xxunk xxunk xxmaj xxunk xxmaj xxunk xxunk xxunk a xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk . xxmaj the xxunk xxunk xxunk xxunk xxunk to xxunk xxunk xxunk xxunk to xxunk the xxmaj xxunk xxmaj xxunk and the xxmaj xxunk xxmaj xxunk xxunk and a xxunk xxunk xxunk xxunk to xxunk the"
2,"the xxunk xxmaj xxunk xxmaj xxunk and xxmaj xxunk xxmaj xxunk xxmaj xxunk xxmaj xxunk xxunk xxunk xxmaj xxunk xxmaj xxunk and xxmaj xxunk xxmaj xxunk xxunk xxunk in xxunk , xxunk xxunk xxunk . xxmaj xxunk xxunk xxunk xxunkxx xxmaj xxunk , a xxunk xxunk xxunk xxunk of xxmaj xxunk xxunk , xxunk in xxunk xxunk xxunk , xxmaj xxunk xxunk xxmaj xxunk xxunk xxmaj xxunk xxunkxx , xxunk"
3,"xxunk . xxmaj xxunk xxunk xxunk to xxunk the xxunk xxunk and xxunk xxunk xxunk xxunk xxunk xxunk . xxmaj xxunk xxmaj xxunk xxunkxx , xxunk xxunk xxmaj xxunk , xxunk in xxunk xxunk xxunk xxunk a xxunk of the xxunk , xxunk xxunk xxunk xxunk the xxmaj xxunk xxunk xxunk xxunk xxunk xxunk of xxunk , and xxunk xxunk xxunk of xxunk xxunk to xxunk xxunk xxunk xxunk xxunk"
4,"xxunk xxunk xxunk xxunk xxunk xxunk xxunk and xxunk xxunk xxunk xxunk xxunk xxunk xxunk . xxmaj xxunk xxunk of xxunk xxunk xxunk , a xxunk xxunk , xxunk xxunk xxunk , xxunk xxunk , xxunk xxunk , xxunk xxunk and xxunk xxunk . xxmaj xxunk the xxunk xxunk xxunk xxunk xxunk , xxunk xxunk , a xxunk xxunk , a xxunk , xxunkxx xxunk , and xxunk . xxmaj"


In [7]:
# epoch = 1
# lr = 1e-3
momentum = (0.8,0.7)
weight_decay = 1.2e-6

In [8]:
vocab = data.train_ds.vocab
vocab_sz = len(vocab.itos); vocab_sz

17

In [9]:
from fastai.callbacks import RNNTrainer
class TransformerTrainer(LearnerCallback):
    def __init__(self, learn, bptt:int, adjust:bool=True):
        super().__init__(learn)
        self.bptt,self.adjust = bptt,adjust
        
    def on_train_begin(self, **kwargs):
        "IN LM, put the training dataloader `first` attribute to `True` to avoid OOM."
        if hasattr(self.learn.data.train_dl, 'first'):
            self.learn.data.first = True
            
    def on_batch_begin(self, last_input, last_target, train, **kwargs):
        last_target = torch.zeros_like(last_target)
        last_input = torch.zeros_like(last_input)
#         last_target = last_input
#         print(last_input.sha)
#         position_ids = torch.arange(last_input.shape[-1], dtype=torch.long, device=last_input.device)
#         position_ids = position_ids.unsqueeze(0).expand_as(last_input)
#         target_ids = torch.arange(last_target.shape[-1], dtype=torch.long, device=last_target.device)
#         target_ids = target_ids.unsqueeze(0).expand_as(last_target)


        position_ids = torch.ones_like(last_input)
        target_ids = torch.ones_like(last_target)
        
        ## STUPID TEST TO SEE IF IT CAN EVEN PREDICT ITSELF
#         return (last_input, position_ids, last_target, target_ids), last_target[:, :-1]
#         return (last_input, position_ids, last_target, target_ids), last_target[:, 1:]
        return (last_input, position_ids, last_target, target_ids), last_target[:, 1:]
    
    def on_backward_begin(self, last_loss:Rank0Tensor, last_input:Tensor, **kwargs):
        "Adjusts the lr to the sequence length and applies AR and TAR to `last_loss`."
        if self.adjust: self.learn.opt.lr *= last_input.size(1) / self.bptt
        return last_loss


In [10]:
import transformer.Constants as Constants
# from dataset import TranslationDataset, paired_collate_fn
from transformer.Models import Transformer
from transformer.Optim import ScheduledOptim

In [11]:
transformer = Transformer(
    n_src_vocab=vocab_sz,
    n_tgt_vocab=vocab_sz,
    len_max_seq=data.bptt+data.max_len,
#     len_max_seq=opt.max_token_seq_len,
#     tgt_emb_prj_weight_sharing=False,
#     emb_src_tgt_weight_sharing=False,
#     d_k=opt.d_k,
#     d_v=opt.d_v,
#     d_model=opt.d_model,
#     d_word_vec=opt.d_word_vec,
#     d_inner=opt.d_inner_hid,
#     n_layers=opt.n_layers,
#     n_head=opt.n_head,
#     dropout=opt.dropout
)

In [22]:
transformer.encoder.src_word_emb(torch.Tensor([0]).long().cuda())

tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan

In [12]:
learn = LanguageLearner(data, transformer, 
                        bptt=bptt, clip=0.1)




In [13]:
rt = learn.callbacks[0]
learn.callbacks = [
    TransformerTrainer(learn, bptt=rt.bptt, adjust=rt.adjust)
]

In [14]:
learn.unfreeze()

In [15]:
learn.fit_one_cycle(22, 1e-3, moms=(0.7,0.5))

epoch,train_loss,valid_loss,accuracy


KeyboardInterrupt: 

In [ ]:
learn.recorder.plot_lr()

In [ ]:
learn.recorder.plot_losses()

In [17]:
learn.save('transformer_run1_1en4');